# FIT Reader - Flexible and Interoperable Data Transfer

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

import fitdecode
import warnings

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class FitReader(BaseReader):
    '''FIT file - Flexible and Interoperable Data Transfer'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)

        self.frames = []


    def load(self):
        '''Load file into memory'''

        counts = \
        {
            'timestamp': 0,
            'position_lat': 0,
            'position_long': 0,
            'distance': 0,
            'altititude': 0,
            'speed': 0,
            'vertical_speed': 0,
            'enhanced_altitude': 0,
            'enhanced_speed': 0,
            'Sat': 0,
            'hdop': 0,
            'cog': 0
        }

        # COROS tracks tend to generate noisy warnings, hence the filter to ignore them!
        # UserWarning: 'field "native_field_num" (idx #0) not found in message "field_description"'
        # (local_mesg_num: 0; chunk_offset: 198); adding dummy dev data...
        with fitdecode.FitReader(self.filename, error_handling=fitdecode.ErrorHandling(0)) as fit:
            for frame in fit:
                if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                    if frame.name == 'record':

                        # Only interested in records with latitude + longitude
                        if frame.has_field('position_lat') and frame.get_value('position_lat'):
                            # Store frame for later
                            self.frames.append(frame)

                            # Update field counts
                            for item in counts:
                                if frame.has_field(item) and frame.get_value(item):
                                    counts[item] += 1

                    elif frame.name in ['device_info', 'file_id']:
                        self.readHeader(frame)

        # Ensure firmware is a string with the correct number of decimals
        if 'firmware' in self.header:
            if 'manufacturer' in self.header and self.header['manufacturer'] == 'garmin':
                self.header['firmware'] = '{:0.2f}'.format(self.header['firmware'])
            else:
                self.header['firmware'] = '{}'.format(self.header['firmware'])

        self.numRecords = len(self.frames)
        if self.numRecords > 0:
            self.consumePoints(counts)


    def readHeader(self, frame):
        '''Read header frame'''

        headerFields = {
            'device': ['product_name'],
            'firmware': ['software_version'],
            'manufacturer': ['manufacturer'],
            'product': ['product', 'garmin_product'],
            'serial': ['serial_number']
        }

        # Garmin files contain a lot of similar frames but the relevant ones all contain a serial number
        if frame.has_field('manufacturer') and frame.get_value('manufacturer') and (
                frame.get_value('manufacturer') != 'garmin' or \
                frame.has_field('serial_number') and frame.get_value('serial_number')):
            for headerField in headerFields:
                for fitField in headerFields[headerField]:
                    if frame.has_field(fitField):
                        self.header[headerField] = frame.get_value(fitField)
                        # Timex Ironman has trailing whitespace in product name
                        if isinstance(self.header[headerField], str):
                            self.header[headerField] = self.header[headerField].strip()

    def consumePoints(self, counts):
        '''Consume points - convert to standard data structure'''

        # Convert timestamp from Garmin to Unix
        iterable = (frame.get_value('timestamp').timestamp() for frame in self.frames)
        self.data['ts'] = np.fromiter(iterable, np.double)
        
        # Convert latitude from semicricles to degrees
        iterable = (frame.get_value('position_lat') * (180 / 2 ** 31) for frame in self.frames)
        self.data['lat'] = np.fromiter(iterable, np.double)
        
        # Convert longitude from semicricles to degrees
        iterable = (frame.get_value('position_long') * (180 / 2 ** 31) for frame in self.frames)
        self.data['lon'] = np.fromiter(iterable, np.double)
            
        # Copy distance, if present
        if counts['distance'] > 0:
            iterable = (0 if frame.has_field('distance') == False or frame.get_value('distance') is None \
                        else frame.get_value('distance') for frame in self.frames)
            self.data['dist'] = np.fromiter(iterable, np.single)

        # Copy elevation, if present
        if counts['enhanced_altitude'] > 0:
            iterable = (0 if frame.has_field('enhanced_altitude') == False or frame.get_value('enhanced_altitude') is None \
                        else frame.get_value('enhanced_altitude') for frame in self.frames)
            self.data['ele'] = np.fromiter(iterable, np.single)

        # Copy speed, if present
        iterable = (0 if frame.has_field('enhanced_speed') == False or frame.get_value('enhanced_speed') is None \
                    else frame.get_value('enhanced_speed') for frame in self.frames)
        self.data['sog'] = np.fromiter(iterable, np.double)
            
        # Copy ROC (rate of climb), if present
        if counts['vertical_speed'] > 0:
            iterable = (0 if frame.has_field('vertical_speed') == False or frame.get_value('vertical_speed') is None \
                        else frame.get_value('vertical_speed') for frame in self.frames)
            self.data['roc'] = np.fromiter(iterable, np.single)

        # Copy COG, if present - e.g. COROS APEX Pro / VERTIX / VERTIX 2
        if counts['cog'] > 0:
            iterable = (0 if frame.has_field('cog') == False or frame.get_value('cog') is None \
                        else frame.get_value('cog') for frame in self.frames)
            self.data['cog'] = np.fromiter(iterable, np.single)

        # Copy satellites, if present - e.g. COROS APEX Pro / VERTIX / VERTIX 2
        if counts['Sat'] > 0:
            iterable = (0 if frame.has_field('Sat') == False or frame.get_value('Sat') is None \
                        else frame.get_value('Sat') for frame in self.frames)
            self.data['sat'] = np.fromiter(iterable, np.uint8)

        # Copy HDOP, if present - e.g. COROS APEX Pro / VERTIX / VERTIX 2
        if counts['hdop'] > 0:
            iterable = (0 if frame.has_field('hdop') == False or frame.get_value('hdop') is None \
                        else frame.get_value('hdop') for frame in self.frames)
            self.data['hdop'] = np.fromiter(iterable, np.uint8)

## COROS APEX Pro Tests

In [3]:
class TestCorosHeader(unittest.TestCase):
    '''Class to test COROS header was correctly loaded'''

    def testDevice(self):
        '''Test the device is as expected'''

        self.assertEqual(corosReader.header['device'], 'COROS APEX Pro')


    def testManufacturer(self):
        '''Test the manufacturer is as expected'''

        self.assertEqual(corosReader.header['manufacturer'], 'coros')


    def testProduct(self):
        '''Test the product is as expected'''

        self.assertEqual(corosReader.header['product'], 841)

In [4]:
class TestCorosData(unittest.TestCase):
    '''Class to test COROS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(corosReader.numRecords, 3809)

        for fieldName in corosReader.data:
            self.assertEqual(corosReader.data[fieldName].size, 3809)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        # Note that this such be 0.7 to 1.6 but the FIT definition breaks the parsing
        self.assertEqual(corosReader.data['hdop'].min(), 0)
        self.assertEqual(corosReader.data['hdop'].max(), 1)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(corosReader.data['sat'].min(), 0)
        self.assertEqual(corosReader.data['sat'].max(), 14)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(corosReader.data['ts'].min(), 1649672017.0)
        self.assertEqual(corosReader.data['ts'].max(), 1649677777.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(np.round(corosReader.data['lat'].min(), 7), 50.5705695)
        self.assertEqual(np.round(corosReader.data['lat'].max(), 7), 50.5832841)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(np.round(corosReader.data['lon'].min(), 7), -2.4620381)
        self.assertEqual(np.round(corosReader.data['lon'].max(), 7), -2.4559603)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(corosReader.data['cog'].min(), 0.0)
        self.assertEqual(corosReader.data['cog'].max(), 359.0)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(corosReader.data['sog'].min(), 0.0)
        self.assertEqual(corosReader.data['sog'].max(), 16.872)

## Garmin Fenix 3 Tests

In [5]:
class TestFenix3Header(unittest.TestCase):
    '''Class to test Fenix 3 header was correctly loaded'''

    def testManufacturer(self):
        '''Test the manufacturer is as expected'''

        self.assertEqual(fenix3Reader.header['manufacturer'], 'garmin')


    def testProduct(self):
        '''Test the product is as expected - derived by fitdecode'''

        self.assertEqual(fenix3Reader.header['product'], 'fenix3')


    def testSerial(self):
        '''Test the serial is as expected'''

        self.assertEqual(fenix3Reader.header['serial'], 3907780652)


    def testFirmware(self):
        '''Test the firmware is as expected'''

        self.assertEqual(fenix3Reader.header['firmware'], '8.20')

In [6]:
class TestFenix3Data(unittest.TestCase):
    '''Class to test Fenix 3 data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(fenix3Reader.numRecords, 11604)

        for fieldName in fenix3Reader.data:
            self.assertEqual(fenix3Reader.data[fieldName].size, 11604)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(fenix3Reader.data['ts'].min(), 1575252722.0)
        self.assertEqual(fenix3Reader.data['ts'].max(), 1575264329.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(np.round(fenix3Reader.data['lat'].min(), 7), -33.2587233)
        self.assertEqual(np.round(fenix3Reader.data['lat'].max(), 7), -33.2389579)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(np.round(fenix3Reader.data['lon'].min(), 7), 151.5426887)
        self.assertEqual(np.round(fenix3Reader.data['lon'].max(), 7), 151.5476529)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(np.round(fenix3Reader.data['ele'].min() - 159.2, 3), 0)
        self.assertEqual(np.round(fenix3Reader.data['ele'].max() - 353.8, 3), 0)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(fenix3Reader.data['sog'].min(), 0.0)
        self.assertEqual(fenix3Reader.data['sog'].max(), 21.041)

## Garmin Fenix 6 Tests

In [7]:
class TestFenix6Header(unittest.TestCase):
    '''Class to test Fenix 6 header was correctly loaded'''

    def testManufacturer(self):
        '''Test the manufacturer is as expected'''

        self.assertEqual(fenix6Reader.header['manufacturer'], 'garmin')


    def testProduct(self):
        '''Test the product is as expected - derived by fitdecode'''

        self.assertEqual(fenix6Reader.header['product'], 'fenix6x')


    def testSerial(self):
        '''Test the serial is as expected'''

        self.assertEqual(fenix6Reader.header['serial'], 3356094893)


    def testFirmware(self):
        '''Test the firmware is as expected'''

        self.assertEqual(fenix6Reader.header['firmware'], '20.30')

In [8]:
class TestFenix6Data(unittest.TestCase):
    '''Class to test Fenix 6 data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(fenix6Reader.numRecords, 8520)

        for fieldName in fenix6Reader.data:
            self.assertEqual(fenix6Reader.data[fieldName].size, 8520)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(fenix6Reader.data['ts'].min(), 1644144393.0)
        self.assertEqual(fenix6Reader.data['ts'].max(), 1644152927.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(np.round(fenix6Reader.data['lat'].min(), 7), 53.3645866)
        self.assertEqual(np.round(fenix6Reader.data['lat'].max(), 7), 53.3714789)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(np.round(fenix6Reader.data['lon'].min(), 7), -3.1908654)
        self.assertEqual(np.round(fenix6Reader.data['lon'].max(), 7), -3.1859821)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(np.round(fenix6Reader.data['ele'].min() + 36, 3), 0)
        self.assertEqual(np.round(fenix6Reader.data['ele'].max() + 3.4, 3), 0)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(fenix6Reader.data['sog'].min(), 0.0)
        self.assertEqual(fenix6Reader.data['sog'].max(), 23.093)

## Suunto Ambit 3 Tests

In [9]:
class TestAmbit3Header(unittest.TestCase):
    '''Class to test Ambit 3 header was correctly loaded'''

    def testManufacturer(self):
        '''Test the manufacturer is as expected'''

        self.assertEqual(ambit3Reader.header['manufacturer'], 'suunto')


    def testDevice(self):
        '''Test the device is as expected - derived by fitdecode'''

        self.assertEqual(ambit3Reader.header['device'], 'Suunto Ambit3 Sport')


    def testProduct(self):
        '''Test the product is as expected - derived by fitdecode'''

        self.assertEqual(ambit3Reader.header['product'], 23)

In [10]:
class TestAmbit3Data(unittest.TestCase):
    '''Class to test Ambit 3 data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(ambit3Reader.numRecords, 509)

        for fieldName in ambit3Reader.data:
            self.assertEqual(ambit3Reader.data[fieldName].size, 509)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(ambit3Reader.data['ts'].min(), 1627572732.0)
        self.assertEqual(ambit3Reader.data['ts'].max(), 1627575512.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(np.round(ambit3Reader.data['lat'].min(), 7), 53.6615636)
        self.assertEqual(np.round(ambit3Reader.data['lat'].max(), 7), 53.6664171)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(np.round(ambit3Reader.data['lon'].min(), 7), -1.0164487)
        self.assertEqual(np.round(ambit3Reader.data['lon'].max(), 7), -1.0130443)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(ambit3Reader.data['ele'].min(), 0)
        self.assertEqual(ambit3Reader.data['ele'].max(), 15)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(ambit3Reader.data['sog'].min(), 0.0)
        self.assertEqual(ambit3Reader.data['sog'].max(), 11.79)


    def testRoc(self):
        '''Test the rate of climb is as expected'''

        self.assertEqual(np.round(ambit3Reader.data['roc'].min() + 0.16, 3), 0)
        self.assertEqual(np.round(ambit3Reader.data['roc'].max() - 0.31, 3), 0)

## Timex Ironman Tests

In [11]:
class TestIronmanHeader(unittest.TestCase):
    '''Class to test Ironman header was correctly loaded'''

    def testManufacturer(self):
        '''Test the manufacturer is as expected'''

        self.assertEqual(ironmanReader.header['manufacturer'], 'timex')


    def testDevice(self):
        '''Test the device is as expected - derived by fitdecode'''

        self.assertEqual(ironmanReader.header['device'], 'Timex Ironman GPS')


    def testProduct(self):
        '''Test the product is as expected - derived by fitdecode'''

        self.assertEqual(ironmanReader.header['product'], 87)

In [12]:
class TestIronmanData(unittest.TestCase):
    '''Class to test Ironman data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(ironmanReader.numRecords, 2618)

        for fieldName in ironmanReader.data:
            self.assertEqual(ironmanReader.data[fieldName].size, 2618)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(ironmanReader.data['ts'].min(), 1651610788.0)
        self.assertEqual(ironmanReader.data['ts'].max(), 1651616155.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(np.round(ironmanReader.data['lat'].min(), 7), 46.0216641)
        self.assertEqual(np.round(ironmanReader.data['lat'].max(), 7), 46.0293621)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(np.round(ironmanReader.data['lon'].min(), 7), -118.9625573)
        self.assertEqual(np.round(ironmanReader.data['lon'].max(), 7), -118.9427841)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(ironmanReader.data['ele'].min(), 100)
        self.assertEqual(ironmanReader.data['ele'].max(), 147)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(ironmanReader.data['sog'].min(), 0.0)
        # TODO - fuse frames containing speed + enhanced_speed with frames containing latitude + longitude
        #self.assertEqual(ironmanReader.data['sog'].max(), 8.48)

## Run Tests

In [13]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.fit')
    corosReader = FitReader(filename)

    filename = os.path.join(projdir, 'sessions', '20191202', '9C2D0835.fit')
    fenix3Reader = FitReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220206', '8249369153_ACTIVITY.fit')
    fenix6Reader = FitReader(filename)

    filename = os.path.join(projdir, 'sessions', '20210729', '6102ee2d2cce8c64d5e98f55.fit')
    ambit3Reader = FitReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220503', '134527CO.fit')
    ironmanReader = FitReader(filename)

    pc1 = time.perf_counter()
    corosReader.load()
    fenix3Reader.load()
    fenix6Reader.load()
    ambit3Reader.load()
    ironmanReader.load()
    pc2 = time.perf_counter()
    
    print("\nFIT files loaded in in %0.2f seconds" % (pc2 - pc1))


FIT files loaded in in 5.43 seconds


In [14]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..................................................
----------------------------------------------------------------------
Ran 50 tests in 0.028s

OK
